In [1]:
from functools import partial
from pathlib import Path

import astroplan as ap
import astropy.units as u

from astropaul.database import database_connection, html_path
import astropaul.targetlistcreator as tlc
import astropaul.lbt as lbt
import astropaul.html as html
import astropaul.phase as ph
import astropaul.priority as pr

%load_ext autoreload
%autoreload 2


In [2]:
date_ranges = [
    ("2026-02-17", "2026-02-20"),
    ("2026-03-11", "2026-03-15"),
    ("2026-04-08", "2026-04-12"),
    ("2026-05-03", "2026-05-10"),
    ("2026-05-24", "2026-06-01"),
    ("2026-06-11", "2026-06-17"),
    ("2026-06-24", "2026-06-29"),
]

session_names = [f"LBT Observing {utc_start}" for utc_start, _ in date_ranges]
html_paths = [(html_path() / name) for name in session_names]
for path in html_paths:
    html.clear_directory(path)

science_target_types = {"QuadEB", "SextEB"}
calibration_target_types = {"RV Standard", "Telluric Standard"}

phase_event_defs = [
    ph.PhaseEventDef("T", partial(ph.calc_time_of_phase, phase=0.0)),
    ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.05)),
    ph.PhaseEventDef("R", partial(ph.calc_time_of_phase, phase=0.18)),
    ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.32)),
    ph.PhaseEventDef("T", partial(ph.calc_time_of_phase, phase=0.45)),
    ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.55)),
    ph.PhaseEventDef("R", partial(ph.calc_time_of_phase, phase=0.68)),
    ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.82)),
    ph.PhaseEventDef("T", partial(ph.calc_time_of_phase, phase=0.95)),
]

pepsi_args = {
    "fiber": "300",
    "cd_blue": 3,
    "cd_red": 6,
    "snr": 100,
    "binocular": True,
    "priority": "(see grid)",
}

readme_name = "UVa_Multistar"
readme_header = Path(f"{readme_name}.header").read_text()
readme_footer = Path(f"{readme_name}.footer").read_text()

science_target_steps = [
    tlc.add_targets,
    partial(tlc.filter_targets, criteria=lambda df: df["Target Type"].isin(science_target_types)),
    tlc.add_lists,
    partial(tlc.filter_targets, criteria=lambda df: df["List LBT 2026A"]),
    tlc.ancillary_data_from_tess,
    partial(tlc.add_database_table, table_name="Ephemerides"),
    tlc.tabulate_ephemerides,
    partial(tlc.add_database_table, table_name="PEPSI Observations"),
    partial(tlc.filter_targets, criteria=lambda df: df["Num PEPSI Observations"] > 0),
    partial(tlc.add_observability),
    partial(lbt.add_pepsi_params, **dict({"num_exp": 2}, **pepsi_args)),
]

calibration_target_steps = [
    tlc.add_targets,
    partial(tlc.filter_targets, criteria=lambda df: df["Target Type"].isin(calibration_target_types)),
    tlc.ancillary_data_from_tess,
    partial(tlc.add_columns_from_sql, table_name="RV Calibration Targets", primary_cols=["RV"]),
    partial(tlc.filter_targets, inverse=True, criteria=lambda df: df["Teff"].isna()),
    partial(tlc.add_observability, time_resolution=2 * u.hour),
    partial(lbt.add_pepsi_params, **dict({"num_exp": 1}, **pepsi_args)),
]

common_steps = [
    partial(tlc.filter_targets, inverse=True, criteria=lambda df: df["Teff"].isna()),
    partial(tlc.filter_targets, criteria=lambda df: (df["Observable Any Night"])),
    tlc.add_tess_catalog_associations,
    partial(tlc.filter_targets, criteria=lambda df: df["PEPSI exp_time"] < 600),
]

target_lists, sessions, csv_tables, readmes = [], [], [], []
for (utc_start, utc_end), name, path in zip(date_ranges, session_names, html_paths):
    session = tlc.ObservingSession(ap.Observer.at_site("LBT"))
    session.add_day_range(utc_start, utc_end)
    sessions.append(session)

    with database_connection() as conn:
        creator = tlc.TargetListCreator(connection=conn, phase_event_defs=phase_event_defs, observing_session=session)
        science_targets = creator.calculate(name="Science Targets", steps=science_target_steps + common_steps)
        calibration_targets = creator.calculate(name="Calibration Targets", steps=calibration_target_steps + common_steps, verbose=False)
        tl = tlc.TargetList.union([science_targets, calibration_targets], name=name)
        tl = creator.calculate(initial_list=tl, steps=[partial(lbt.assign_rv_standards, target_types=science_target_types)])
    tl.target_list["PEPSI notes"] = tl.target_list["Target Type"]
    csv_table, readme = lbt.write_lbt_readme_file(readme_header, readme_footer, tl.target_list, session)
    csv_tables.append(csv_table.to_csv(index=False))
    readmes.append(readme)
    target_lists.append(tl)

    print(tl.summarize())

Name: LBT Observing 2026-02-17
Criteria:
  Union of lists: Science Targets and Calibration Targets:
    List Science Targets:
      All targets loaded (732 targets remain)
      lambda df: df["Target Type"].isin({'SextEB', 'QuadEB'}) (322 targets remain)
      lambda df: df["List LBT 2026A"] (22 targets remain)
      lambda df: df["Num PEPSI Observations"] > 0 (22 targets remain)
      Observability calculated at LBT in 15.0 min intervals from 2026-02-17 to 2026-02-20
        AltitudeConstraint: {'min': 30.0, 'max': 80.0, 'boolean_constraint': True}
      Inverse of: lambda df: df["Teff"].isna() (22 targets remain)
      lambda df: (df["Observable Any Night"]) (22 targets remain)
      lambda df: df["PEPSI exp_time"] < 600 (22 targets remain)
    List Calibration Targets:
      All targets loaded (732 targets remain)
      lambda df: df["Target Type"].isin({'RV Standard', 'Telluric Standard'}) (394 targets remain)
      Inverse of: lambda df: df["Teff"].isna() (392 targets remain)
    

In [3]:
illumination_categories = [
    ((0.0, 0.4), "Dark"),
    ((0.4, 0.7), "Gray"),
    ((0.7, 1.0), "Bright"),
]

distance_categories = {
    "Dark": [
        ((0, 180), 1),
    ],
    "Gray": [
        ((0, 5), 0.1),
        ((5, 15), 0.85),
        ((15, 180), 1),
    ],
    "Bright": [
        ((0, 15), 0.25),
        ((15, 30), 0.75),
        ((30, 180), 1),
    ],
}

altitude_categories = [
    ((-90, 35), 0),
    ((35, 45), 0.95),
    ((45, 90), 1),
]

phase_scores = {
    "B|B": 0.5,
    "B|R": 0.8,
    "B|T": 0.4,
    "R|B": 0.8,
    "R|R": 0.9,
    "R|T": 1,
    "T|B": 0.4,
    "T|R": 1,
    "T|T": 0.1,
}

ephem_categories = [
    ((3.5, 99), 1),
    ((2.1, 3.5), .9),
    ((0, 2.1), .7),
]

for (utc_start, _), tl, session, html_path, csv_table, readme in zip(date_ranges, target_lists, sessions, html_paths, csv_tables, readmes):
    science_tl = tl.copy()
    science_tl.target_list = science_tl.target_list[science_tl.target_list["Target Type"].isin(["QuadEB", "SextEB"])]
    pl = pr.PriorityList(science_tl, session, interval=30 * u.min)
    pr.calculate_moon_priority(pl, illumination_categories=illumination_categories, dist_categories=distance_categories)
    pr.calculate_altitude_priority(pl, altitude_categories=altitude_categories)
    pr.calculate_ephemerides_priority(pl, categories=ephem_categories)
    pr.calculate_list_priority(pl, "HQND", false_value=0.75)
    pr.calculate_phase_priority(pl, phase_defs=phase_event_defs, phase_categories=phase_scores)
    pr.calculate_overall_priority(pl)
    pr.aggregate_target_priorities(pl, skip_column_threshold=0.3)
    pl.categorize_priorities(
        bins=[0.00, 0.20, 0.40, 0.6, 1.00], labels=["", "*", "* *", "* * *"]
    )
    (Path(utc_start) / f"{readme_name}.csv").write_text(csv_table)
    (Path(utc_start) / f"{readme_name}.readme").write_text(readme)
    other_files = {
        "LBT Readme": readme,
        "LBT CSV": csv_table,
    }
    html.render_observing_pages(tl, pl, other_files, html_path)
    print(f"Wrote results for {utc_start} to {html_path}")

 '2026-02-17T03:00:00.000000000' '2026-02-17T03:30:00.000000000'
 '2026-02-17T04:00:00.000000000' '2026-02-17T04:30:00.000000000'
 '2026-02-17T05:00:00.000000000' '2026-02-17T05:30:00.000000000'
 '2026-02-17T06:00:00.000000000' '2026-02-17T06:30:00.000000000'
 '2026-02-17T07:00:00.000000000' '2026-02-17T07:30:00.000000000'
 '2026-02-17T08:00:00.000000000' '2026-02-17T08:30:00.000000000'
 '2026-02-17T09:00:00.000000000' '2026-02-17T09:30:00.000000000'
 '2026-02-17T10:00:00.000000000' '2026-02-17T10:30:00.000000000'
 '2026-02-17T11:00:00.000000000' '2026-02-17T11:30:00.000000000'
 '2026-02-17T12:00:00.000000000' '2026-02-17T12:30:00.000000000'
 '2026-02-17T13:00:00.000000000' '2026-02-17T13:30:00.000000000'], obsgeoloc=[( 2120783.02697874,  4942635.10572476, 3422155.48134876),
 ( 1455733.8943113 ,  5176546.60108431, 3423845.01549722),
 (  765790.92518845,  5321403.08690306, 3425601.26401   ),
 (   62823.87105504,  5374712.45839225, 3427394.01371043),
 ( -641073.45081197,  5335557.5836526

Wrote results for 2026-02-17 to \Users\User\Dropbox\Astro\Observing Files\LBT Observing 2026-02-17


 '2026-03-11T03:00:00.000000000' '2026-03-11T03:30:00.000000000'
 '2026-03-11T04:00:00.000000000' '2026-03-11T04:30:00.000000000'
 '2026-03-11T05:00:00.000000000' '2026-03-11T05:30:00.000000000'
 '2026-03-11T06:00:00.000000000' '2026-03-11T06:30:00.000000000'
 '2026-03-11T07:00:00.000000000' '2026-03-11T07:30:00.000000000'
 '2026-03-11T08:00:00.000000000' '2026-03-11T08:30:00.000000000'
 '2026-03-11T09:00:00.000000000' '2026-03-11T09:30:00.000000000'
 '2026-03-11T10:00:00.000000000' '2026-03-11T10:30:00.000000000'
 '2026-03-11T11:00:00.000000000' '2026-03-11T11:30:00.000000000'
 '2026-03-11T12:00:00.000000000' '2026-03-11T12:30:00.000000000'
 '2026-03-11T13:00:00.000000000'], obsgeoloc=[(  145206.73974244,  5373253.16092323, 3427181.93940025),
 ( -559207.96078537,  5344891.22892018, 3428984.51177543),
 (-1253851.25198687,  5224578.11035038, 3430765.39290111),
 (-1926772.51787578,  5014383.66724348, 3432493.94412136),
 (-2566394.84732456,  4717924.07636203, 3434140.42690435),
 (-3161714

Wrote results for 2026-03-11 to \Users\User\Dropbox\Astro\Observing Files\LBT Observing 2026-03-11


 '2026-04-08T03:00:00.000000000' '2026-04-08T03:30:00.000000000'
 '2026-04-08T04:00:00.000000000' '2026-04-08T04:30:00.000000000'
 '2026-04-08T05:00:00.000000000' '2026-04-08T05:30:00.000000000'
 '2026-04-08T06:00:00.000000000' '2026-04-08T06:30:00.000000000'
 '2026-04-08T07:00:00.000000000' '2026-04-08T07:30:00.000000000'
 '2026-04-08T08:00:00.000000000' '2026-04-08T08:30:00.000000000'
 '2026-04-08T09:00:00.000000000' '2026-04-08T09:30:00.000000000'
 '2026-04-08T10:00:00.000000000' '2026-04-08T10:30:00.000000000'
 '2026-04-08T11:00:00.000000000' '2026-04-08T11:30:00.000000000'
 '2026-04-08T12:00:00.000000000' '2026-04-08T12:30:00.000000000'], obsgeoloc=[(-2359394.25012758,  4825140.65906454, 3433625.92125242),
 (-2970527.053389  ,  4473683.84558468, 3435204.86277428),
 (-3530403.8552275 ,  4045264.01020164, 3436655.1006977 ),
 (-4029392.55693112,  3547251.67061623, 3437951.68404791),
 (-4458908.57624204,  2988214.61120676, 3439072.30521183),
 (-4811562.5361684 ,  2377770.48320721, 343

...

Wrote results for 2026-05-24 to \Users\User\Dropbox\Astro\Observing Files\LBT Observing 2026-05-24


 '2026-06-11T04:00:00.000000000' '2026-06-11T04:30:00.000000000'
 '2026-06-11T05:00:00.000000000' '2026-06-11T05:30:00.000000000'
 '2026-06-11T06:00:00.000000000' '2026-06-11T06:30:00.000000000'
 '2026-06-11T07:00:00.000000000' '2026-06-11T07:30:00.000000000'
 '2026-06-11T08:00:00.000000000' '2026-06-11T08:30:00.000000000'
 '2026-06-11T09:00:00.000000000' '2026-06-11T09:30:00.000000000'
 '2026-06-11T10:00:00.000000000' '2026-06-11T10:30:00.000000000'
 '2026-06-11T11:00:00.000000000' '2026-06-11T11:30:00.000000000'], obsgeoloc=[(-5200321.51590366, -1324090.7990454 , 3441213.48676629),
 (-4982200.47852721, -1994486.75447171, 3440670.2184879 ),
 (-4678213.58505824, -2630567.88347259, 3439904.19014955),
 (-4293590.61362968, -3221391.07194902, 3438928.58002425),
 (-3834948.6022507 , -3756791.82161227, 3437760.17219817),
 (-3310178.00954801, -4227559.11960835, 3436419.06782488),
 (-2728306.96762956, -4625593.90441698, 3434928.33930984),
 (-2099345.96245867, -4944048.40178408, 3433313.6333743

Wrote results for 2026-06-11 to \Users\User\Dropbox\Astro\Observing Files\LBT Observing 2026-06-11


 '2026-06-24T04:00:00.000000000' '2026-06-24T04:30:00.000000000'
 '2026-06-24T05:00:00.000000000' '2026-06-24T05:30:00.000000000'
 '2026-06-24T06:00:00.000000000' '2026-06-24T06:30:00.000000000'
 '2026-06-24T07:00:00.000000000' '2026-06-24T07:30:00.000000000'
 '2026-06-24T08:00:00.000000000' '2026-06-24T08:30:00.000000000'
 '2026-06-24T09:00:00.000000000' '2026-06-24T09:30:00.000000000'
 '2026-06-24T10:00:00.000000000' '2026-06-24T10:30:00.000000000'
 '2026-06-24T11:00:00.000000000' '2026-06-24T11:30:00.000000000'], obsgeoloc=[(-4776926.37145776, -2446358.40985781, 3440176.67374493),
 (-4415563.87553246, -3051688.20997938, 3439259.41222162),
 (-3978083.61236969, -3604515.16070575, 3438143.65663034),
 (-3472011.97481125, -4095328.44928578, 3436848.60208116),
 (-2906055.3989912 , -4515684.14355978, 3435396.52849366),
 (-2289950.57915254, -4858350.46103503, 3433812.41729787),
 (-1634296.95794518, -5117432.18418551, 3432123.52165577),
 ( -950374.37403333, -5288472.08154168, 3430358.8975972

Wrote results for 2026-06-24 to \Users\User\Dropbox\Astro\Observing Files\LBT Observing 2026-06-24


In [4]:
# import asyncio
# import glob
# # for categorical_file in glob.glob("*.*"):

# for categorical_file in glob.glob("../../Observing Files/LBT Observing 2025-07-04/Categorical Priorities *.html"):

#     await html.html_to_pdf(categorical_file, categorical_file.replace(".html", ".pdf"))
#     print(categorical_file)